In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers as L
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from imblearn.under_sampling import RandomUnderSampler


In [2]:
path = r'C:\Luna_CS\Melanoma 2020\train.csv'
root_path = r'C:\Luna_CS\Melanoma 2020'
train_path = os.path.join(root_path, 'train')
full_df = pd.read_csv(path)
full_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,tfrecord,width,height
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,0,6000,4000
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,0,6000,4000
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,6,1872,1053
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,0,1872,1053
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,11,6000,4000


In [5]:
short_df = {}
short_df['image_name'] = [name+'.jpg' for name in full_df['image_name']]
short_df['label'] = [1.0 if lab == 'malignant' else 0.0 for lab in full_df['benign_malignant']]
short_df = pd.DataFrame(short_df)
###Random Undersampling###
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(np.array(short_df['image_name']).reshape(-1, 1), 
                                np.array(short_df['label']).reshape(-1, 1))
new_short_df = {}
new_short_df['image_name'] = X_res[:, 0]
new_short_df['label'] = y_res[:]#[lab for lab in y_res]
##########################
new_short_df = pd.DataFrame(new_short_df)
new_short_df.head()
# new_short_df

,image_name,label
0,ISIC_9085719.jpg,0.0
1,ISIC_3990124.jpg,0.0
2,ISIC_5125438.jpg,0.0
3,ISIC_3076237.jpg,0.0
4,ISIC_5209007.jpg,0.0


In [9]:
import h5py

full_img_arr = []
for img_name in new_short_df['image_name']:
    img_path = os.path.join(train_path, img_name)
    img = load_img(img_path, target_size=(256, 256))
    img_arr = img_to_array(img)/255.
    full_img_arr.append(img_arr)
    
full_img_arr = np.array(full_img_arr)



In [10]:
hf = h5py.File('short_data.h5', 'w')

hf.create_dataset('images', data=full_img_arr)
hf.create_dataset('labels', data=new_short_df['label'])

hf.close()